In [ ]:
# default_exp agent

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Agent

In [ ]:
from bfh_mt_hs2020_rl_basics.env import CarEnv

import gym
import ptan
import torch
import torch.nn as nn

In [ ]:
#export
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x.float())

In [ ]:
#export
from bfh_mt_hs2020_rl_basics.env import CarEnv

import gym
import ptan

class Agent:
    
    def __init__(self, env: CarEnv, gamma:float, buffer_size:int, eps_start:float = 1.0, eps_final:float = 0.02, eps_frames:int = 10**5, target_net_sync:int = 1000):
        
        self.env = env
        self.target_net_sync = target_net_sync
        
        self.net = self._config_net()
        
        self.tgt_net = ptan.agent.TargetNet(self.net)
        
        self.selector = ptan.actions.EpsilonGreedyActionSelector(
                                    epsilon=1, 
                                    selector=ptan.actions.ArgmaxActionSelector())
        
        self.epsilon_tracker = ptan.actions.EpsilonTracker(selector=self.selector, eps_start=eps_start, eps_final=eps_final, eps_frames=eps_frames)

        self.agent = agent = ptan.agent.DQNAgent(self.net, self.selector)
        
        self.exp_source = ptan.experience.ExperienceSourceFirstLast(self.env, self.agent, gamma=gamma)
        self.buffer = ptan.experience.ExperienceReplayBuffer(self.exp_source, buffer_size=buffer_size)
        

    def _config_net(self)-> nn.Module:
        return Net(self.env.observation_space.shape[0], 128, self.env.action_space.n)
    
    
    def iteration_completed(self, iteration: int):
        
        self.epsilon_tracker.frame(iteration)
        
        if iteration % self.target_net_sync == 0:
            self.tgt_net.sync()

        

In [ ]:
GAMMA = 0.9
REPLAY_SIZE = 1000

In [ ]:
def test_agent():
    env = CarEnv()
    agent = Agent(env, gamma=GAMMA, buffer_size=REPLAY_SIZE)

In [ ]:
test_agent()